In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import pandas as pd
from helper import get_news_dataset,get_data, create_classes

pd.options.display.max_columns = None

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import LogisticRegression

# Preprocessing libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from math import sqrt

# MAKE SURE THE DATASETS ARE CORRECT

 ![caption](files/requirements.png)

## Import DataSet

In [8]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
news_url  = "https://www.dropbox.com/sh/euppz607r6gsen2/AACq4aMWDOIw2I_SSGqJ-r2Oa/Online%20News%20Popularity%20(Mashable%20News)?dl=1"
housing_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAD6JGlvG5XADIjg9SCojvpya/House%20Sales%20in%20King%20County%2C%20USA?dl=1&preview=kc_house_data.csv"
all_urls = [sumdata_url,news_url,housing_url]

In [9]:
get_data(all_urls) # retrieves the data if there is NO data folder

In [10]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv" 
housing_price_path ="data/kc_house_data.csv" 

## Load datasets sum_noise

In [11]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";") 
sumdata_noise.head(n=2)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,62485,58472,84200,86181,75529,136939,150633,230058,246491,257336,1352179,Very Large Number
1,2,75559,119137,146760,139674,19582,177083,217746,321110,434444,516798,1976446,Very Large Number


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [12]:
# Use 'Nosiy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise['Noisy Target'].values.reshape(-1, 1)
le = LabelEncoder() 
# Use 'Nosiy Target Class' Large Number as regression target 
sumdata_noise_classif_Y = le.fit_transform(sumdata_noise['Noisy Target Class'])

# Use rest columns as explananatory variables 
sumdata_noise_X = sumdata_noise.iloc[:, 1:-2].values
# first column is instance (just a row number), and the last two features are target class and target
 
scX = StandardScaler() #standardising features
scY = StandardScaler() # standardising y 
sumdata_noise_X = scX.fit_transform(sumdata_noise_X)
sumdata_noise_reg_Y = scY.fit_transform(sumdata_noise_reg_Y)


/home/jibin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


## Load datasets sumdata

In [13]:
sumdata = pd.read_csv(sumdata_path, delimiter=";")  
sumdata.head(n=2)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Target,Target Class
0,1,57326,68791,82549,99059,72624,142645,171174,205409,246491,295789,1369233,Very Large Number
1,2,87859,105431,126517,151820,19982,218621,262345,314814,377777,453332,2098516,Very Large Number


## Preprocess sumdata dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Target' as regression target
- Extract 'Target Class' as classification target
- Extract rest of the columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [14]:
# Use 'Nosiy Target' as regression target
sumdata_reg_Y = sumdata['Target'].values.reshape(-1, 1)

# Use 'Nosiy Target Class' Large Number as classification target
le = LabelEncoder() 
sumdata_classif_Y = le.fit_transform(sumdata['Target Class'])

# Use rest columns as explananatory variables 
sumdata_X = sumdata.iloc[:, 1:-2].values

 

scX = StandardScaler() #standardising features
scY = StandardScaler() # standardising y 
sumdata_X      = scX.fit_transform(sumdata_X)
sumdata_reg_Y  = scX.fit_transform(sumdata_reg_Y)




/home/jibin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


 ## Load House Price dataset

In [15]:
housing_price = pd.read_csv(housing_price_path)

# Remove 'Instance' as it simply represents the row number
housing_price = housing_price.drop('id', axis = 1)
housing_price.head(n=3)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


## Preprocess housing price dataset

- Remove 'Id' as it simply represents the row number
- Use 'SalePrice' as regression target
- Use 'SaleCondition' as classification target
- For explananatory variables, use the following numerical variable and categorical variables
    - Numerical
        - LotFrontage
        - OverallQual
        - OverallCond
        - YearBuilt
        - YearRemodAdd
        - TotalBsmtSF
        - GrLivArea
        - FullBath
        - TotRmsAbvGrd
        - GarageYrBlt
        - GarageCars
        - GarageArea
        - LotArea
        - 1stFlrSF
        - 2ndFlrSF
    - Categorical
        - YearSold
        - Neighborhood
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [16]:
## filter unused categorical columns and drop NaN, we only want to keep 'neighborhood' and 'housestyle' 
## in our explanantory vairbale

# get all the numerical data that is needed
explanatory_numeric = ['OverallQual', 'LotFrontage', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', 'GrLivArea',
                      'FullBath', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'LotArea', '1stFlrSF', '2ndFlrSF']

housing_price[explanatory_numeric[0]]
filtered_table = housing_price[explanatory_numeric[0]]
filtered_table

for c in explanatory_numeric[1:]:
    filtered_table = pd.concat([filtered_table, housing_price[c]], axis=1)
filtered_table

# get the two categorical variable that we wants to use, convert it to dummy variable and avoid dummy variable trap
filtered_table = pd.concat([filtered_table, pd.get_dummies(housing_price['Neighborhood']).iloc[:, :-1]], axis=1)
filtered_table = pd.concat([filtered_table, pd.get_dummies(housing_price['YrSold']).iloc[:, :-1]], axis=1)

# drop NaN
filtered_table = filtered_table.dropna()
filtered_table 

# get explanatory variable
housing_price_reg_X = filtered_table[:].values
housing_price_classif_X = filtered_table[:].values

# get the regression target and classification target
# I am using sales price as both regression and classification target
# sale_price_classi = SalePrice<Mean, SalePrice>=Mean
housing_price_reg_Y = housing_price['SalePrice'].values
housing_price_classif_Y = np.zeros(housing_price_reg_Y.shape)
house_price_mean = housing_price_reg_Y.mean()

for i, price in enumerate(housing_price_reg_Y):
    if price >= house_price_mean:
        housing_price_classi_Y[i] = 1
    else:
        housing_price_classi_Y[i] = 0

# Apply Feature Scaling to the numeric variables and regression target 
scX = StandardScaler()
scY = StandardScaler()
housing_price_classif_X = scX.fit_transform(housing_price_classif_X)
housing_price_classif_Y = scY.fit_transform(housing_price_classif_Y.reshape(-1, 1))




KeyError: 'OverallQual'

## News dataset

In [18]:
news = get_news_dataset() 
news.head(n=1)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.1,0.7,-0.35,-0.6,-0.2,0.5,-0.1875,0.0,0.1875,593


## Preprocess news dataset
- Below are the features  that will be selected 
-

In [19]:
columns= [ ' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
       ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos',
       ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle',
       ' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min',
       ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg',
       ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares',
       ' self_reference_max_shares', ' self_reference_avg_sharess', ' global_subjectivity', ' global_rate_positive_words',
       ' global_rate_negative_words', ' rate_positive_words',
       ' rate_negative_words', ' avg_positive_polarity',
       ' min_positive_polarity', ' max_positive_polarity',' title_subjectivity', ' abs_title_subjectivity',
       ' abs_title_sentiment_polarity', ' shares']

#### Classes
- 0 - 10            class 1
- 10 - 100          class 2 
- 101 - 1000        class 3
- 1001 - 10,000     class 4
- 10,001 - 100,000  class 5
- 100,001 -         class 6

In [20]:
news = news[columns] 
# Classification the target values 
news['label'] = news.apply(create_classes, axis=1)


In [21]:
news_reg_Y = news[' shares'].values.reshape(-1, 1)

news_reg_X = news.iloc[:, 0:-2].values # the last 2 columns are the target values 
# no need to label encode as they already encoded
news_classif_Y = news['label'] 

scX = StandardScaler() #standardising features
scY = StandardScaler() # standardising y 
news_reg_X  = scX.fit_transform(news_reg_X)
news_reg_Y  = scX.fit_transform(news_reg_Y)
 

/home/jibin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


# Fits Algorithms to datasets

In [22]:
data_chunks = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000,
1000000, 5000000, 10000000, 50000000, 100000000]

In [23]:

def root_mean_square_error(y_actual, y_predicted):
    return sqrt(mean_squared_error(y_actual, y_predicted))

In [24]:
def model( X, y, dataset_name, algorithm, isReg, algo_params={}): 
    
    print ("Algorithm: {}\nDataset: {}\n".format( algorithm.__name__, dataset_name))
    for chunk in data_chunks:
        
        # if chunk is greater than the no. of examples quite from the chunking
        if chunk > X.shape[0]: 
            break
        
        print ("Chunk Size: {}\n".format(chunk))
        
        # generate the chunk file
        current_X = X[0:chunk]
        current_y = y[0:chunk]
        
        kFoldModelling(current_X, current_y, 10, algorithm, isReg, algo_params)
        
         
         
    

In [25]:
def kFoldModelling (X, y, kfolds, algorithm, isReg, algo_params):
    
    kf = KFold(n_splits=kfolds, shuffle=True)
    rmse = np.zeros((10,1))
    mae = np.zeros((10,1))
    accuracy = np.zeros((10,1))
    precision = np.zeros((10,1))
    
    
    for i, (train_index, test_index) in enumerate(kf.split(X)):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
      
        # fit the model to the datset
        lm = algorithm(**algo_params)
        lm.fit(X_train, y_train)
        
        if isReg:     
            rmse[i] = root_mean_square_error(y_test, lm.predict(X_test))  # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
            mae[i] = mean_absolute_error(y_test, lm.predict(X_test))
        else:
            from sklearn.metrics import accuracy_score
            accuracy[i] = accuracy_score(y_test, lm.predict(X_test))
            
            precision[i] = precision_score(y_test, lm.predict(X_test),average="weighted")
        # print the result, we will need to have method that genereates the result csv file required.
    
    if isReg:     
        print ("Iteration: {}\nRMSE: {}\nMAE: {}\n".format( i, rmse.mean(), mae.mean()))
    else:
        print ("Iteration: {}\Accuracy: {}\nPrecision: {}\n".format( i, accuracy.mean(), precision.mean()))
    

## Fits Regression Algorithms to datasets

    - Linear Regression
    - Random Forest Regression
    

In [27]:
model(sumdata_noise_X, sumdata_noise_reg_Y, "The Sum Dataset(with noise)", LinearRegression, True)
model(sumdata_X, sumdata_reg_Y, "The Sum Dataset(without noise)",  LinearRegression, True) 

# kwards = {}

model(news_reg_X, news_reg_Y, "News dataset", LinearRegression, True)


# model(sumdata_noise_X, sumdata_noise_reg_Y,"The Sum Dataset(with noise)",  RandomForestRegressor, True)
# model(sumdata_X, sumdata_reg_Y, "The Sum Dataset(without noise)", RandomForestRegressor, True) 
# model(news_reg_X, news_reg_Y, "News dataset", RandomForestRegressor, True)



Algorithm: LinearRegression
Dataset: The Sum Dataset(with noise)

Chunk Size: 100

Iteration: 9
RMSE: 0.1373363236238759
MAE: 0.10484380848305759

Chunk Size: 500

Iteration: 9
RMSE: 0.12504502619649216
MAE: 0.09450578564533159

Chunk Size: 1000

Iteration: 9
RMSE: 0.11888088517979549
MAE: 0.08959336121375767

Chunk Size: 5000

Iteration: 9
RMSE: 0.11961380693448001
MAE: 0.0897502521489513

Chunk Size: 10000

Iteration: 9
RMSE: 0.12005203798392824
MAE: 0.09036892315857734

Chunk Size: 50000

Iteration: 9
RMSE: 0.11976028123419499
MAE: 0.08962692999653196

Chunk Size: 100000

Iteration: 9
RMSE: 0.12023159205023601
MAE: 0.09001686031780014

Chunk Size: 500000

Iteration: 9
RMSE: 0.11994529572947518
MAE: 0.08982487349265202

Chunk Size: 968135

Iteration: 9
RMSE: 0.11984228429482599
MAE: 0.08971960669080573

Algorithm: LinearRegression
Dataset: The Sum Dataset(without noise)

Chunk Size: 100

Iteration: 9
RMSE: 3.845945191815846e-16
MAE: 3.1606661732297423e-16

Chunk Size: 500

Iteration:

KeyboardInterrupt: 

### Classification algorithms

In [28]:
LogisticRegression_params = {"multi_class":"multinomial","solver":"lbfgs" }
model(sumdata_noise_X, sumdata_noise_classif_Y, "The Sum Dataset(with noise)", LogisticRegression, False,LogisticRegression_params)
#model(sumdata_X, sumdata_classif_Y, "The Sum Dataset(without noise)", LogisticRegression, False)
#model(news_reg_X, news_classif_Y, "News dataset", LogisticRegression, False)
 
# from sklearn.svm import SVC
# model(sumdata_noise_classi_X, sumdata_noise_classif_Y, "The Sum Dataset(with noise)", SVC, False)
# model(sumdata_classi_X, sumdata_classif_Y, "The Sum Dataset(without noise)", SVC, False)
# model(housing_price_classi_X, housing_price_classif_Y, "Housing Dataset", SVC, False)
# model(titanic_classification_X, titanic_classification_y, "Titanic Dataset", SVC, False)


Algorithm: LogisticRegression
Dataset: The Sum Dataset(with noise)

Chunk Size: 100

Iteration: 9\Accuracy: 0.96
Precision: 0.9723611111111111

Chunk Size: 500

Iteration: 9\Accuracy: 0.9560000000000001
Precision: 0.9376923929808436

Chunk Size: 1000



/home/jibin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Iteration: 9\Accuracy: 0.9569999999999999
Precision: 0.9441114097596813

Chunk Size: 5000

Iteration: 9\Accuracy: 0.9667999999999999
Precision: 0.9658374659996829

Chunk Size: 10000

Iteration: 9\Accuracy: 0.9667
Precision: 0.9657350159972701

Chunk Size: 50000

Iteration: 9\Accuracy: 0.96792
Precision: 0.9663661815011789

Chunk Size: 100000

Iteration: 9\Accuracy: 0.96752
Precision: 0.9658440775770973

Chunk Size: 500000

Iteration: 9\Accuracy: 0.967208
Precision: 0.9654891687988718

Chunk Size: 968135



KeyboardInterrupt: 